In [7]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn


epochs = 50
trainable=True

In [8]:
from keras_contrib.layers import CRF


ModuleNotFoundError: No module named 'keras_contrib'

In [106]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [107]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [108]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [164]:
train_sentences = get_sentences('/srv/Resources/GermEVAL/NER-de-train.tsv')
dev_sentences = get_sentences('/srv/Resources/GermEVAL/NER-de-dev.tsv')
test_sentences = get_sentences('/srv/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [167]:
print(train_sentences[0])

[['Schartau', ['S', 'c', 'h', 'a', 'r', 't', 'a', 'u'], 'B-PER'], ['sagte', ['s', 'a', 'g', 't', 'e'], 'O'], ['dem', ['d', 'e', 'm'], 'O'], ['"', ['"'], 'O'], ['Tagesspiegel', ['T', 'a', 'g', 'e', 's', 's', 'p', 'i', 'e', 'g', 'e', 'l'], 'B-ORG'], ['"', ['"'], 'O'], ['vom', ['v', 'o', 'm'], 'O'], ['Freitag', ['F', 'r', 'e', 'i', 't', 'a', 'g'], 'O'], [',', [','], 'O'], ['Fischer', ['F', 'i', 's', 'c', 'h', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['"', ['"'], 'O'], ['in', ['i', 'n'], 'O'], ['einer', ['e', 'i', 'n', 'e', 'r'], 'O'], ['Weise', ['W', 'e', 'i', 's', 'e'], 'O'], ['aufgetreten', ['a', 'u', 'f', 'g', 'e', 't', 'r', 'e', 't', 'e', 'n'], 'O'], [',', [','], 'O'], ['die', ['d', 'i', 'e'], 'O'], ['alles', ['a', 'l', 'l', 'e', 's'], 'O'], ['andere', ['a', 'n', 'd', 'e', 'r', 'e'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['überzeugend', ['ü', 'b', 'e', 'r', 'z', 'e', 'u', 'g', 'e', 'n', 'd'], 'O'], ['war', ['w', 'a', 'r'], 'O'], ['"', ['"'], 'O'], ['.', ['.'], 'O']]


In [111]:
train_sentences = modify_labels(train_sentences)
dev_sentences = modify_labels(dev_sentences)
test_sentences = modify_labels(test_sentences)

In [166]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [147]:
labelSet = set()
words = {}
characters=set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
            words[token.lower()] = True

In [148]:
print(labelSet)

{'I-ORGpart', 'B-PERpart', 'I-ORG', 'B-LOCpart', 'B-ORGpart', 'B-ORGderiv', 'B-LOCderiv', 'I-LOC', 'I-PERpart', 'I-LOCpart', 'I-OTH', 'I-OTHderiv', 'I-OTHpart', 'I-LOCderiv', 'B-LOC', 'O', 'B-OTHderiv', 'B-PERderiv', 'I-PER', 'I-PERderiv', 'B-ORG', 'B-PER', 'I-ORGderiv', 'B-OTH', 'B-OTHpart'}


In [149]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [150]:
print(label2Idx)

{'I-ORGpart': 0, 'O': 15, 'B-OTHderiv': 16, 'B-PERderiv': 17, 'I-PER': 18, 'I-PERderiv': 19, 'B-PERpart': 1, 'I-ORG': 2, 'B-ORG': 20, 'B-PER': 21, 'B-ORGpart': 4, 'B-ORGderiv': 5, 'B-LOCderiv': 6, 'I-PERpart': 8, 'I-LOC': 7, 'I-LOCpart': 9, 'I-ORGderiv': 22, 'I-OTH': 10, 'B-LOCpart': 3, 'B-OTH': 23, 'B-OTHpart': 24, 'I-OTHderiv': 11, 'I-OTHpart': 12, 'I-LOCderiv': 13, 'B-LOC': 14}


In [151]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [152]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/srv/fastText/german_words.txt','wb')
f.write(string_words.encode())
f.close()

In [153]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'allLower': 1, 'mainly_numeric': 5, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'numeric': 0, 'allUpper': 2, 'initialUpper': 3}


In [154]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('german_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [155]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[2]))

300


In [156]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'Å': 0, 'x': 91, 'Ø': 244, 'f': 1, 'ğ': 326, 'Е': 248, 'π': 5, '台': 158, '≤': 6, 'o': 7, '柯': 10, '±': 11, '貴': 12, '³': 13, 'ъ': 14, 'ῦ': 2, '妃': 104, 'オ': 15, 'б': 16, 'ŏ': 26, 'õ': 25, '(': 20, '©': 216, '9': 22, 'ë': 24, '\x99': 51, '«': 27, 'l': 28, 'ü': 99, 'з': 45, ';': 29, '術': 324, 'а': 325, '*': 48, 'd': 30, 'O': 112, 'æ': 31, 'š': 32, 'ʻ': 102, 'т': 33, '佐': 34, 'υ': 281, '+': 35, '“': 36, 'ă': 38, 'Ш': 290, 'Ц': 268, 'm': 3, 'Â': 40, '章': 41, 'ó': 42, '傳': 43, 'Þ': 163, 'я': 17, 'ť': 44, 't': 46, '7': 47, 'C': 49, 'η': 56, '\x96': 52, 'ņ': 53, 'Č': 55, 'Ä': 54, '九': 58, '대': 266, 'И': 60, '동': 61, '£': 62, ')': 63, 'Ş': 64, '⊃': 9, 'є': 4, 'ض': 117, '5': 8, 'Ü': 65, 'ż': 67, 'в': 225, 'г': 68, 'İ': 69, 'v': 70, 'ب': 323, 'á': 72, '寝': 181, 'Æ': 73, 'ä': 74, 'î': 75, '루': 81, 'D': 77, 'z': 78, 'σ': 80, '2': 82, 'й': 83, '−': 84, 'Λ': 85, '¸': 86, 'Z': 286, ']': 87, 'c': 88, 'γ': 92, 'Ö': 90, 'ḳ': 94, '算': 101, '&': 97, 'и': 98, '鷹': 100, '冲': 103, '`': 21, 'Π': 230, 'ν': 10

In [132]:
print(trainSentences[0]) 

[['Schartau', ['S', 'c', 'h', 'a', 'r', 't', 'a', 'u'], 'B-PER'], ['sagte', ['s', 'a', 'g', 't', 'e'], 'O'], ['dem', ['d', 'e', 'm'], 'O'], ['"', ['"'], 'O'], ['Tagesspiegel', ['T', 'a', 'g', 'e', 's', 's', 'p', 'i', 'e', 'g', 'e', 'l'], 'B-ORG'], ['"', ['"'], 'O'], ['vom', ['v', 'o', 'm'], 'O'], ['Freitag', ['F', 'r', 'e', 'i', 't', 'a', 'g'], 'O'], [',', [','], 'O'], ['Fischer', ['F', 'i', 's', 'c', 'h', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['"', ['"'], 'O'], ['in', ['i', 'n'], 'O'], ['einer', ['e', 'i', 'n', 'e', 'r'], 'O'], ['Weise', ['W', 'e', 'i', 's', 'e'], 'O'], ['aufgetreten', ['a', 'u', 'f', 'g', 'e', 't', 'r', 'e', 't', 'e', 'n'], 'O'], [',', [','], 'O'], ['die', ['d', 'i', 'e'], 'O'], ['alles', ['a', 'l', 'l', 'e', 's'], 'O'], ['andere', ['a', 'n', 'd', 'e', 'r', 'e'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['überzeugend', ['ü', 'b', 'e', 'r', 'z', 'e', 'u', 'g', 'e', 'n', 'd'], 'O'], ['war', ['w', 'a', 'r'], 'O'], ['"', ['"'], 'O'], ['.', ['.'], 'O']]


In [157]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [158]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
# print((train_set[0][2]))

[[78469, 57810, 34228, 42986, 15727, 42986, 18854, 46322, 2432, 38832, 22811, 42986, 8922, 77072, 67337, 42036, 2432, 54196, 57356, 57720, 32119, 36161, 31908, 42986, 21278], [3, 1, 1, 4, 3, 4, 1, 3, 4, 3, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 1, 1, 4, 4], array([[ 18,  88, 288, ...,   0,   0,   0],
       [173, 236, 256, ...,   0,   0,   0],
       [ 30,  76,   3, ...,   0,   0,   0],
       ...,
       [188, 236, 187, ...,   0,   0,   0],
       [146,   0,   0, ...,   0,   0,   0],
       [295,   0,   0, ...,   0,   0,   0]], dtype=int32), [21, 15, 15, 15, 20, 15, 15, 15, 15, 21, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]]
25


In [159]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [160]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 9840        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_13 (TimeDistri (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [161]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch
        labels = labels.tolist()
#         print(labels)
        for sentence in labels:
            for i,label in enumerate(sentence):
                temp = [0]*len(label2Idx)
                value = label[0]
                temp[value]=1
                sentence[i] = temp
        labels = np.array(labels)
#         print(labels)
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
#         print('*')
        print(' ')

Epoch 0/50
51/51 [==============================] - 43s 847ms/step
 
55/51 [================================] - 43s 788ms/step
 
Epoch 1/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 680ms/step
 
55/51 [================================] - 35s 632ms/step
 
Epoch 2/50
51/51 [==============================] - 35s 681ms/step
 
55/51 [================================] - 35s 633ms/step
 
Epoch 3/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 710ms/step
 
55/51 [================================] - 36s 660ms/step
 
Epoch 4/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 685ms/step
 
55/51 [================================] - 35s 637ms/step
 
Epoch 5/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 698ms/step
 
55/51 [================================] - 36s 649ms/step
 
Epoch 6/50
 2/51 [

51/51 [==============================] - 35s 685ms/step
 
55/51 [================================] - 35s 637ms/step
 
Epoch 7/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 687ms/step
 
55/51 [================================] - 35s 639ms/step
 
Epoch 8/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 706ms/step
 
55/51 [================================] - 36s 656ms/step
 
Epoch 9/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 691ms/step
 
55/51 [================================] - 35s 643ms/step
 
Epoch 10/50
51/51 [==============================] - 36s 714ms/step
 
55/51 [================================] - 37s 664ms/step
 
Epoch 11/50
51/51 [==============================] - 35s 686ms/step
 
55/51 [================================] - 35s 638ms/step
 
Epoch 12/50
 2/51 [>.............................] - ETA: 0s 
 
34/51 [========

51/51 [==============================] - 35s 692ms/step
 
55/51 [================================] - 35s 644ms/step
 
Epoch 13/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 713ms/step
 
55/51 [================================] - 36s 662ms/step
 
Epoch 14/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 688ms/step
 
55/51 [================================] - 35s 639ms/step
 
Epoch 15/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 700ms/step
 
55/51 [================================] - 36s 650ms/step
 
Epoch 16/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 688ms/step
 
55/51 [================================] - 35s 639ms/step
 
Epoch 17/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 711ms/step
 
55/51 [=========================

 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 692ms/step
 
55/51 [================================] - 35s 643ms/step
 
Epoch 20/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 37s 724ms/step
 
55/51 [================================] - 37s 672ms/step
 
Epoch 21/50
51/51 [==============================] - 35s 693ms/step
 
55/51 [================================] - 35s 644ms/step
 
Epoch 22/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 37s 718ms/step
 
55/51 [================================] - 37s 667ms/step
 
Epoch 23/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 33s 641ms/step
 
55/51 [================================] - 33s 596ms/step
 
Epoch 24/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 32s 626ms/step
 
55/51 [=========================

51/51 [==============================] - 33s 648ms/step
 
55/51 [================================] - 33s 602ms/step
 
Epoch 26/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 32s 632ms/step
 
55/51 [================================] - 32s 588ms/step
 
Epoch 27/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 33s 648ms/step
 
55/51 [================================] - 33s 603ms/step
 
Epoch 28/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 32s 631ms/step
 
55/51 [================================] - 32s 587ms/step
 
Epoch 29/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 37s 719ms/step
 
55/51 [================================] - 37s 668ms/step
 
Epoch 30/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 33s 647ms/step
 
55/51 [=========================

51/51 [==============================] - 32s 628ms/step
 
55/51 [================================] - 32s 585ms/step
 
Epoch 32/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 33s 644ms/step
 
55/51 [================================] - 33s 599ms/step
 
Epoch 33/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 34s 665ms/step
 
55/51 [================================] - 34s 618ms/step
 
Epoch 34/50
51/51 [==============================] - 35s 690ms/step
 
55/51 [================================] - 35s 642ms/step
 
Epoch 35/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 715ms/step
 
55/51 [================================] - 37s 665ms/step
 
Epoch 36/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 689ms/step
 
55/51 [================================] - 35s 640ms/step
 
Epoch 37/50
 2/51 [>....

51/51 [==============================] - 35s 683ms/step
 
55/51 [================================] - 35s 635ms/step
 
Epoch 39/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 685ms/step
 
55/51 [================================] - 35s 636ms/step
 
Epoch 40/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 708ms/step
 
55/51 [================================] - 36s 658ms/step
 
Epoch 41/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 692ms/step
 
55/51 [================================] - 35s 644ms/step
 
Epoch 42/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 711ms/step
 
55/51 [================================] - 36s 661ms/step
 
Epoch 43/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 685ms/step
 
55/51 [=========================

51/51 [==============================] - 36s 709ms/step
 
55/51 [================================] - 36s 659ms/step
 
Epoch 45/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 684ms/step
 
55/51 [================================] - 35s 636ms/step
 
Epoch 46/50
51/51 [==============================] - 35s 688ms/step
 
55/51 [================================] - 35s 640ms/step
 
Epoch 47/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 704ms/step
 
55/51 [================================] - 36s 654ms/step
 
Epoch 48/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 35s 685ms/step
 
55/51 [================================] - 35s 636ms/step
 
Epoch 49/50
 2/51 [>.............................] - ETA: 0s 
 
51/51 [==============================] - 36s 705ms/step
 
55/51 [================================] - 36s 655ms/step
 


In [162]:
predLabels, correctLabels = tag_dataset(dev_batch)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.806, Rec: 0.789, F1: 0.797


In [26]:
predLabels, correctLabels = tag_dataset(dev_batch)  
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))

2197/2199 [============================>.] - ETA: 0s0.4401060032720164
0.9647900855029301
0.9616731407603598


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [163]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
pre_test, rec_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_test, rec_test, f1_test))

5097/5099 [============================>.] - ETA: 0sTest-Data: Prec: 0.797, Rec: 0.763, F1: 0.780


In [140]:
id2word = {v: k for k, v in word2Idx.items()}
print(len(correctLabels))

5099


In [141]:
print(idx2Label)

{0: 'I-ORGpart', 1: 'B-PERpart', 2: 'I-ORG', 3: 'B-LOCpart', 4: 'B-ORGpart', 5: 'B-ORGderiv', 6: 'B-LOCderiv', 7: 'I-LOC', 8: 'I-PERpart', 9: 'I-LOCpart', 10: 'I-OTH', 11: 'I-OTHderiv', 12: 'I-OTHpart', 13: 'I-LOCderiv', 14: 'B-LOC', 15: 'O', 16: 'B-OTHderiv', 17: 'B-PERderiv', 18: 'I-PER', 19: 'I-PERderiv', 20: 'B-ORG', 21: 'B-PER', 22: 'I-ORGderiv', 23: 'B-OTH', 24: 'B-OTHpart'}


In [142]:
id = 1
f = open('test.txt','wb')
for sentence_number, batch in enumerate(test_batch):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
        string = string.encode()
        f.write(string)
        id+=1
        

In [271]:
predLabels, correctLabels = tag_dataset(test_batch)  
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))

5097/5099 [============================>.] - ETA: 0s0.716534749169935
0.9633095985821336
0.9595560770928965


In [164]:
model.save('german_ner.h5')

In [165]:
print(len(word2Idx.keys()))

89975


In [166]:
print(len(trainSentences)+len(testSentences)+len(devSentences))

31297
